# CycleAtlanta GPS Cleaning
This notebook is for cleaning the GPS traces from CycleAtlanta. It's based on the
GPS_Clean_Route_Snapping.sql script that Dr. Aditi Misra used.

Overview
1. Imports and combines all of the coords.csv files (in the future we should just work from the original sql)
1. Add column names and format data
1. Remove trips with fewer than 5 minutes of data recorded (modifiable)
1. Create a trip dataframe to cross reference with trip.csv (only keep trips that are in both)
1. Drop trip duplicates
1. Turn coords df into dictionary
1. Remove points with unrealisic jumps (> 47mph)
1. With the unrealistic ones removed, only consider trip inside the specified area (inside the i-285 perimeter for this study)
1. Split trips that have a pause of 5 mins or more
1. Export the coords and trips into a pkl file (replace with a .db in the future)

In [1]:
import pandas as pd
import geopandas as gpd
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
import pickle
from pathlib import Path
from tqdm import tqdm
import numpy as np
import datetime
import json

import gps_utils

In [2]:
config = json.load((Path.cwd() / 'config.json').open('rb'))
export_fp = Path(config['project_directory']) / 'CycleAtlanta'
if export_fp.exists() == False:
    export_fp.mkdir()

# Inputs

In [3]:
#filepaths for traces
cycleatl_fp = Path.home() / 'Documents/BikewaySimData/Data/CycleAtlanta'
coords_fps = cycleatl_fp.glob('coord*.csv')

#coordinate reference system to project to
project_crs = config['projected_crs_epsg']

#set the minimum number of points required for a trip to be considered
#300 is based on the number of points needed for a second-by-second trace of five minutes
point_threshold = 5 * 60

# Read in raw gps coordinate files from CycleAtlanta
There are several coordinate files, and the some trips span across them so this
block loads them all into memory (~5 mins for computer with 32GB RAM).

In [4]:
# initialize empty dataframe and dictionary for adding cleaned data
coords = pd.DataFrame()

#add each coords csv to all coords dataframe (only possible if lots of RAM)
#TODO modify to run this step with SQL
# NOTE some trips span across the different CSV files so retain last trip ids somewhere if you have to read
# in one at a time
for coords_fp in coords_fps:
    print('Reading',coords_fp.name)
    one_coords = pd.read_csv(coords_fp,header=None)
    coords = pd.concat([coords,one_coords],ignore_index=True)

# rename columns
col_names = ['tripid','datetime','lat','lon','altitude_m','speed_kph','hAccuracy_m','vAccuracy_m']
coords.columns = col_names

# replace -1 speed with NA
coords.loc[coords['speed_kph']==-1,'speed_kph'] = np.nan

# convert speed and accuracy to imperial units (mph and ft)
coords['speed_mph'] = coords['speed_kph'] * 2.2369362920544025
coords['hAccuracy_ft'] = coords['hAccuracy_m'] * 3.28084 

# drop unneeded ones
coords.drop(columns=['altitude_m','hAccuracy_m','vAccuracy_m','speed_kph'],inplace=True)

# change dates to datetime
coords['datetime'] = pd.to_datetime(coords['datetime'])

# add geometry info and turn into geodataframe
coords['geometry'] = gpd.points_from_xy(coords['lon'],coords['lat'])
coords = gpd.GeoDataFrame(coords,geometry='geometry',crs='epsg:4326')
coords.to_crs(project_crs,inplace=True)

#add X/Y column
coords['X'] = coords.geometry.x
coords['Y'] = coords.geometry.y

# sort everything
coords.sort_values(['tripid','datetime'],inplace=True)

# count and print number of trips
print(f"{coords.shape[0]} total points found")
print(f"{coords['tripid'].nunique()} initial trips found")

Reading coord-1.csv
Reading coord-2.csv
Reading coord-3.csv
Reading coord-4.csv
Reading coord-5.csv
Reading coord.csv
58631361 total points found
28240 initial trips found


# Pre-Cleaning

## Drop duplicate points

In [5]:
dups = coords[['tripid','datetime']].duplicated()
coords = coords[~dups]
print('Dropped',dups.sum(),'duplicate points')
print(f"{coords.shape[0]} total points remaining")

Dropped 10000000 duplicate points
48631361 total points remaining


## Remove trips fewer than 300 points (theoretical max for 5 minute trip with at least 1 sec sampling)
Doing it this way because duration alone does not say anything about the amount of data

In [6]:
below_5 = (coords['tripid'].value_counts() < point_threshold)
coords = coords[~coords['tripid'].isin(below_5[below_5].index.tolist())]
print(below_5.sum(),f'trips had fewer than {point_threshold} points',coords['tripid'].nunique(),'trips remaining')

2737 trips had fewer than 300 points 25503 trips remaining


## Filter to study area

In [7]:
studyarea = gpd.read_file(Path(config['studyarea'])).unary_union
within = coords.clip(studyarea)
within = pd.merge(coords['tripid'].value_counts(),within['tripid'].value_counts(),left_index=True,right_index=True,how='left')

### Completely within study area
completely_within = within[within['tripid_x'] == within['tripid_y']].index.tolist()

### Partially within study area
partial = within[within['tripid_x'] > within['tripid_y']].index.tolist()
print(len(partial),'trips were partially outside of the study area')

### Completely out of study area
no_overlap = within[within['tripid_y'].isna()].index.tolist()
print(len(no_overlap),'trips were completey outside of the study area')

coords = coords[coords['tripid'].isin(completely_within)]
print(coords['tripid'].nunique(),'trips remaining')

841 trips were partially outside of the study area
2586 trips were completey outside of the study area
22076 trips remaining


# Create trips dataframe
These next lines aggregate the points by trip id to find the start/end time/location,
duration, numbner of points, and average hAccuracy. These are recorded as initial values to compare against the cleaned dataset. Then find and remove duplicate trips 

In [8]:
#get start time
start_time = coords.groupby('tripid')['datetime'].min()
start_time.rename('initial_start_time',inplace=True)

#get end time
end_time = coords.groupby('tripid')['datetime'].max()
end_time.rename('initial_end_time',inplace=True)

#get duration
duration = end_time - start_time
duration.rename('initial_duration',inplace=True)

#get starting location unprojected
start_lon = coords.groupby('tripid')['datetime'].idxmin().map(coords['lon'])
start_lat = coords.groupby('tripid')['datetime'].idxmin().map(coords['lat'])
start_lon.rename('initial_start_lon',inplace=True)
start_lat.rename('initial_start_lat',inplace=True)

#get ending location unprojected
end_lon = coords.groupby('tripid')['datetime'].idxmax().map(coords['lon'])
end_lat = coords.groupby('tripid')['datetime'].idxmax().map(coords['lat'])
end_lon.rename('initial_end_lon',inplace=True)
end_lat.rename('initial_end_lat',inplace=True)

#get starting location projected
start_X = coords.groupby('tripid')['datetime'].idxmin().map(coords['X'])
start_Y = coords.groupby('tripid')['datetime'].idxmin().map(coords['Y'])
start_X.rename('initial_start_X',inplace=True)
start_Y.rename('initial_start_Y',inplace=True)

#get ending location projected
end_X = coords.groupby('tripid')['datetime'].idxmax().map(coords['X'])
end_Y = coords.groupby('tripid')['datetime'].idxmax().map(coords['Y'])
end_X.rename('initial_end_X',inplace=True)
end_Y.rename('initial_end_Y',inplace=True)

#get number of points
num_of_points = coords['tripid'].value_counts()
num_of_points.rename('initial_total_points',inplace=True)

#get average haccuracy
avg_accuracy = coords.groupby('tripid')['hAccuracy_ft'].mean()
avg_accuracy.rename('initial_avg_accuracy',inplace=True)

#turn into df
trips_df = pd.concat([start_time,end_time,
                      start_lon,start_lat,end_lon,end_lat,
                      start_X,start_Y,end_X,end_Y,
                      duration,num_of_points,avg_accuracy],axis=1)
trips_df.reset_index(inplace=True)
trips_df.rename(columns={'index':'tripid'},inplace=True)

## Cross reference with the trip table
It appears that there are more trips in the coords data than in the trip csv. However, these are all past 2016, so we'll remove them for now.

Also, the start times in trip.csv do not line up with the coords.csv, but the end times and initial_num_points do. Used start times from coords.

In [9]:
trip = pd.read_csv(cycleatl_fp/'trip.csv',header=None)
col_names = ['tripid','userid','trip_type','description','starttime','endtime','initial_num_points']
trip.columns = col_names
trip.drop(columns=['starttime','endtime','initial_num_points'],inplace=True)
before_merge = trips_df.shape[0]
trips_df = pd.merge(trips_df,trip,on='tripid')
print(before_merge-trips_df.shape[0],'trips are not in trip.csv and have been dropped')

#remove tripids from coords_df
coords = coords[coords['tripid'].isin(set(trips_df['tripid'].tolist()))]
print(trips_df.shape[0],'trips remaining')

9 trips are not in trip.csv and have been dropped
22067 trips remaining


## Find and remove duplicates


In [10]:
duplicates = trips_df.drop(columns=['tripid']).duplicated()
print(f"{duplicates.sum()} trips are duplicates")
duplicate_tripids = trips_df[duplicates]['tripid'].tolist()

1484 trips are duplicates


In [11]:
trips_df = trips_df[~duplicates]
coords = coords[~coords['tripid'].isin(duplicate_tripids)]
print(trips_df.shape[0],'trips remaining')

20583 trips remaining


# Cleaning

## Remove points after pause

In [12]:
pause_threshold_min = 2
coords['time_diff'] = coords.groupby('tripid')['datetime'].apply(lambda x: x.diff())
trips_above_threshold = coords.loc[coords['time_diff'] > datetime.timedelta(minutes=pause_threshold_min),'tripid'].unique().tolist()
print(len(trips_above_threshold),'trips have at least one',pause_threshold_min,'minute pause')

1717 trips have at least one 2 minute pause


In [13]:
def keep_points_before_pause(group):
    
    group = group.copy()

    #get indices of all pauses
    idxs = group[group['time_diff'] > datetime.timedelta(minutes=pause_threshold_min)].index.tolist()

    #go through each index
    for idx in idxs:
        #retrieve the time stamp
        pause_time = group.loc[idx,'datetime']
        
        #count before and after
        before = group['datetime'] < pause_time
        after = group['datetime'] >= pause_time

        # if before.sum() >= after.sum():
        #     return group[before]
        if before.sum() >= point_threshold:
            return group[before]
        else:
            group = group[after]
    
    #if the last group is the only one choose that
    return group

coords = coords.groupby('tripid').apply(keep_points_before_pause).reset_index(drop=True)

In [14]:
pause_threshold_min = 2
coords['time_diff'] = coords.groupby('tripid')['datetime'].apply(lambda x: x.diff())
trips_above_threshold = coords.loc[coords['time_diff'] > datetime.timedelta(minutes=pause_threshold_min),'tripid'].unique().tolist()
print(len(trips_above_threshold),'trips have at least one',pause_threshold_min,'minute pause')

0 trips have at least one 2 minute pause


## Speed Filter
Sustained speeds over 20 mph hint that the trip is probably a car trip.

In [15]:
maxspeed = 30
high_maxspeed = 100
sustained_speeds = coords.groupby('tripid')['speed_mph'].apply(lambda speed: (((speed > maxspeed) & (speed < high_maxspeed)).sum() > 1*60))
sustained_speeds = sustained_speeds[sustained_speeds].index.tolist()
print(len(sustained_speeds),'trips have sustained speeds above',maxspeed,'mph')
coords = coords[~coords['tripid'].isin(sustained_speeds)]
print(coords['tripid'].nunique(),'trips remaining')

153 trips have sustained speeds above 30 mph
20430 trips remaining


## hAccuracy Filter
It's not clear how much this metric helps, but it may help to remove points that are substantially above the mean hAccuracy level for that trip.

In [16]:
z_score = coords.groupby('tripid')['hAccuracy_ft'].apply(lambda group: (((group - group.mean())) / group.std()) > 4)
print(z_score.sum(),'points were 4 standard deviations above the mean hAccuracy')
coords = coords[~z_score]

275100 points were 4 standard deviations above the mean hAccuracy


# Store coords in dictionary and export

In [17]:
coords_dict = {}
coords_dict.update({tripid : df.reset_index(drop=True) for tripid, df in coords.groupby('tripid')})

In [18]:
len(coords_dict.keys())

20430

Some speed values are NULL, but these trips appear to still be good

In [19]:
# #TODO fix this
# #any NA readings?
# coords[coords['speed_mph'].isna()]
# null_speeds = coords.groupby('tripid')['speed_mph'].apply(lambda speed: speed.isna().sum()).sort_values(ascending=False)
# null_speeds
# group_size = coords['tripid'].value_counts()
# group_size.index.name = 'tripid'
# test = pd.concat([null_speeds,group_size],axis=1,ignore_index=False)
# test.sort_values('speed_mph',ascending=False).head(10)
# coords[coords['tripid']==25308].drop(columns=['datetime']).explore()

One more 

In [20]:
gps_count = [[key,item.shape[0]] for key, item in coords_dict.items()]
df = pd.DataFrame(gps_count,columns=['tripid','points'])
remove = df.loc[df['points'] < point_threshold,'tripid'].tolist()

for tripid in remove:
    coords_dict.pop(tripid)

trips_df = trips_df[~trips_df['tripid'].isin(remove)]

In [28]:
len(coords_dict.keys())

20359

In [31]:
print(trips_df.shape[0])

20359


In [30]:
#make sure trips_df and coords_dict match up
trips_df = trips_df[trips_df['tripid'].isin(list(coords_dict.keys()))]

# Caluclate new trip metrics

In [21]:
for tripid, coords in coords_dict.items():
    coords = gps_utils.calculate_coordinate_metrics(coords)
    trips_df = gps_utils.calculate_trip_metrics(tripid,coords,trips_df)

# Export

In [32]:
with (export_fp/'trips_0.pkl').open('wb') as fh:
    pickle.dump(trips_df,fh)

In [23]:
with (export_fp/'coords_0.pkl').open('wb') as fh:
    pickle.dump(coords_dict,fh)

# Deprecated Cleaning Functions

## Remove the first 5 seconds of a trip (in-case the GPS location wasn't immediately triangulated)
Use this format for more complicated cleaning functions.

In [24]:
# def remove_first_seconds(group,seconds):
#     start_time = group.min()
#     after = (group - start_time) > datetime.timedelta(seconds=seconds)
#     return group[after]
# drop_first_seconds = coords.groupby('tripid')['datetime'].apply(lambda group: remove_first_seconds(group,5)).reset_index()['level_1']
# before_drop = coords.shape[0]
# coords = coords.loc[drop_first_seconds]
# print(before_drop-coords.shape[0],'coordinates dropped')

## Speed deviation
Takes 8 mins. Removed because it takes too long, and it doesn't really seem to help.

In [25]:
# remove_list_speed_deviation = []
# speed_deviation_max_mph = 47

# for tripid, coords_df in tqdm(coords_dict.items()):
#     '''
#     This loop searches for unrealistic jumps between gps points
#     and removes these points until there are no more jumps.

#     It first looks at the first point in the data to see if there
#     is a jump after it. This helps remove points that were recorded before
#     the GPS had been triangulated. If the first two points are recorded close together,
#     then this wouldn't help.
    
#     Then it looks at subsequent points and removes them if the threshold is
#     exceeded.
    
#     All the while, trips are removed if this process eliminates too many points
#     '''
    
#     # #calculate metrics for the coordinates dataframe
#     # orig_cols = coords_df.columns
#     # coords_df = gps_utils.calculate_coordinate_metrics(coords_df)
    
#     # if second value has speed above speed_deviation_max_mph remove the first point until no jump is dectected
#     # this should deal with cases in which the first point is way off
#     while coords_df.iloc[1]['speed_mph'] >= speed_deviation_max_mph:
#         coords_df = coords_df.iloc[1:,:]
#         #remove if this gets rid of too many points
#         if coords_df.shape[0] < 3:
#             #remove_list_speed_deviation.append(tripid)
#             break
#         # #recalculate metrics
#         # coords_df = gps_utils.calculate_coordinate_metrics(coords_df)
    
#     if coords_df.shape[0] < point_threshold:
#         remove_list_speed_deviation.append(tripid)
#         continue

#     # remove point if speed_deviation_max_mph between it and previous point
#     while (coords_df['speed_mph']>=speed_deviation_max_mph).sum() > 0:
#         # only keep if below speed_deviation_max_mph mph or it's the first row and has na as the value
#         coords_df = coords_df[(coords_df['speed_mph']<speed_deviation_max_mph) | (coords_df['speed_mph'].isna())]
#         # make sure there are enough points
#         if coords_df.shape[0] < point_threshold:
#             #emove_list_speed_deviation.append(tripid)
#             break
#         #recalculate metrics
#         coords_df = gps_utils.calculate_coordinate_metrics(coords_df)

#     if coords_df.shape[0] < point_threshold:
#         remove_list_speed_deviation.append(tripid)
#         continue

#     # remove extra columns
#     coords_df = coords_df[orig_cols]
    
#     # assign cleaned coords to dictionary
#     coords_dict[tripid] = coords_df

# for key in remove_list_speed_deviation:
#     coords_dict.pop(key)
# trips_df = trips_df[~trips_df['tripid'].isin(remove_list_speed_deviation)]
# print(len(remove_list_speed_deviation),'trips removed after speed deviation filter')
# print(trips_df.shape[0],'trips remaining')

## Trim out trip chains
Identifies GPS trip chains and trim trips.

Some trips will have gaps between point recordings. When the gap is sufficiently large, it
indicates that the person may have forgotten to stop their recording,
stopped at a destination along the way (trip chaining), or that there was some other error.

This algorithm checks for cases in which the person probably forgot to stop
the recording and trims the excess points.

If a chain was detected, then only the first leg of the trip is retained.
There are only a small number of these.

Step 0: Set the pause threshold
Step 1: For each trip, count the number of points that exceed this threshold
Step 2: For the first pause detected, trim trip if all subsequent points are
within pause threshold biking distance (using avg speed of 8 mph)
Step 3: If not all points are contained, then consider these points to be part of a new trip
chain. Repeat step 1 until all legs of trip chains are identfied.
Step 4: Remove trip chains from databases and export them for later examination.

In [26]:
# remove_list_tripchain = []
# pause_threshold_min = 5

# for tripid, coords_df in tqdm(coords_dict.items()):
#     '''
#     Identifies GPS trip chains and trim trips

#     Some trips will have gaps between point recordings. When the gap is large, it
#     indicates that the person may have forgotten to stop their recording,
#     stopped at a destination along the way (trip chaining), or that there was some error.

#     This algorithm checks for cases in which the person probably forgot to stop
#     the recording and trims the excess points.

#     If a chain was detected, then only the first leg of the trip is retained.
#     There are only a small number of these.

#     Step 0: Set the pause threshold
#     Step 1: For each trip, count the number of points that exceed this threshold
#     Step 2: For the first pause detected, trim trip if all subsequent points are
#     within pause threshold biking distance (using avg speed of 8 mph)
#     Step 3: If not all points are contained, then consider these points to be part of a new trip
#     chain. Repeat step 1 until all legs of trip chains are identfied.
#     Step 4: Remove trip chains from databases and export them for later examination.

#     '''
#     #calculate metrics for the coordinates dataframe
#     orig_cols = coords_df.columns
#     coords_df = gps_utils.calculate_coordinate_metrics(coords_df)


    
#     # reset index to ensure subsequent numbers
#     coords_df.reset_index(drop=True,inplace=True)    

#     #TODO split the trip into multiple segments and assign it a new tripid
#     #while (coords_df['delta_time'] > datetime.timedelta(minutes=pause_threshold_min)).any():
#     if (coords_df['delta_time'] > datetime.timedelta(minutes=pause_threshold_min)).any():
        
#         #find first pause
#         first_pause = (coords_df['delta_time'] > datetime.timedelta(minutes=pause_threshold_min)).idxmax()
        
#         #trim trip
#         coords_df = coords_df.loc[0:first_pause]        

#     coords_df = gps_utils.calculate_coordinate_metrics(coords_df)

#     if coords_df.shape[0] < point_threshold:
#         remove_list_tripchain.append(tripid)
#         continue

#     # remove extra columns
#     coords_df = coords_df[orig_cols]
#     # assign cleaned coords to dictionary
#     coords_dict[tripid] = coords_df
# d
# for key in remove_list_tripchain:
#     coords_dict.pop(key)
# trips_df = trips_df[~trips_df['tripid'].isin(remove_list_tripchain)]
# print(len(remove_list_tripchain),'trips removed during trip chain detection')
# print(trips_df.shape[0],'trips remaining')